<a href="https://colab.research.google.com/github/bengsoon/predictive-maintenance/blob/master/Predictive_Maintenance_BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn==0.23.1
!pip install imbalanced-learn==0.7.0

     |████████████████████████████████| 6.8MB 6.9MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 174kB 5.1MB/s 
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


In [ ]:
!pip install plotly --upgrade
!pip install chart-studio --upgrade

     |████████████████████████████████| 13.2MB 228kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 71kB 3.9MB/s 


In [ ]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from sklearn import metrics

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

import types

In [ ]:
from pathlib import Path
import os
import matplotlib.pyplot as plt

In [ ]:
path = Path("/content/drive/MyDrive/Colab_Notebooks/Predictive_Maintenance")
os.listdir(path)

['Predictive_Maintenance.ipynb',
 'ai4i2020.csv',
 'df_for_export-2.csv',
 'equipment_failure_data_2.csv',
 'equipment_failure_data_1.csv',
 '.ipynb_checkpoints',
 'machine_failure_prediction',
 'equipment_failure_data_2.csv.1',
 'equipment_failure_data_1.csv.1',
 'df_with_flipper',
 'data_with_rolling_spikes',
 'Machine Learning for  Equipment Maintenance - Pub.ipynb']

In [ ]:
#Import the second half from github
# os.chdir(path)
# !wget https://raw.githubusercontent.com/shadgriffin/machine_failure/master/equipment_failure_data_2.csv
# !wget https://raw.githubusercontent.com/shadgriffin/machine_failure/master/equipment_failure_data_1.csv

--2021-06-18 03:25:32--  https://raw.githubusercontent.com/shadgriffin/machine_failure/master/equipment_failure_data_2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11762512 (11M) [text/plain]
Saving to: ‘equipment_failure_data_2.csv.1’

equipment_failure_d 100%[===================>]  11.22M  23.4MB/s    in 0.5s    

2021-06-18 03:25:34 (23.4 MB/s) - ‘equipment_failure_data_2.csv.1’ saved [11762512/11762512]

--2021-06-18 03:25:34--  https://raw.githubusercontent.com/shadgriffin/machine_failure/master/equipment_failure_data_1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
H

In [ ]:
df_1 = pd.read_csv(path / "equipment_failure_data_1.csv", parse_dates=True); df_1.head()

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
0,100001,12/2/14,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880
1,100001,12/3/14,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881
2,100001,12/4/14,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882
3,100001,12/5/14,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883
4,100001,12/6/14,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884


In [ ]:
df_1.describe()

,ID,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
count,149855.000000,149855.000000,149855.000000,149855.000000,149855.000000,149855.000000,149855.000000,149855.000000,149855.000000,149855.000000,149855.000000,149855.000000
mean,100320.419512,2.497561,14.185534,85.918460,35.643733,4557.518000,7.925408,8.229842,110.082879,0.001368,117.481750,2751.146341
std,178.503502,1.093775,8.665254,85.572229,14.723678,2497.993986,2.426462,13.041714,197.356230,0.036961,204.107481,3368.354905
min,100001.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-16.490000,0.000000
25%,100164.000000,2.000000,7.548000,0.000000,28.250000,3193.000000,6.596000,0.900000,11.300000,0.000000,8.660000,755.000000
50%,100328.000000,2.000000,11.077500,95.480000,35.020000,4224.000000,8.025600,4.200000,34.000000,0.000000,44.320000,1143.000000
75%,100472.000000,3.000000,22.080000,163.660000,41.940833,5615.000000,9.480700,10.500000,118.900000,0.000000,135.080000,3224.500000
max,100612.000000,4.000000,51.225600,478.470000,570.510000,52767.000000,23.533800,291.100000,3995.900000,1.000000,1824.960000,15170.000000


In [ ]:
print("Min date", df_1["DATE"].min(), ", Max Date: ", df_1["DATE"].max())

Min date 1/1/15 , Max Date:  9/9/16


In [ ]:
df_2 = pd.read_csv(path / "equipment_failure_data_2.csv", parse_dates=True); df_2.head()

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
0,100002,12/2/14,D,L,R,6,23.52,0.0,34.15,4064.0,7.842741,31.1,68.331034,0,0.0,820
1,100002,12/3/14,D,L,R,6,27.36,0.0,34.12,4060.0,8.089900,31.1,68.403571,0,0.0,821
2,100002,12/4/14,D,L,R,6,29.52,0.0,33.86,4029.0,9.657700,31.1,68.444444,0,0.0,822
3,100002,12/5/14,D,L,R,6,22.80,0.0,34.07,4054.0,6.520538,31.1,68.488462,0,0.0,823
4,100002,12/6/14,D,L,R,6,24.24,0.0,34.39,4093.0,9.161052,26.3,68.560000,0,0.0,824


In [ ]:
df_2.describe()

,ID,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
count,157896.000000,157896.000000,157896.000000,157896.000000,157896.000000,157896.000000,157896.000000,157896.000000,157896.000000,157896.000000,157896.000000,157896.000000
mean,100301.722222,6.486111,14.964497,74.900503,34.424619,4788.152427,8.016408,9.865662,164.423424,0.001368,170.465310,2308.796296
std,176.206263,1.134395,8.942271,85.679924,14.153097,2537.695817,2.217307,19.846309,269.865252,0.036961,268.490199,2930.234090
min,100002.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.610000,0.000000
25%,100153.500000,5.000000,7.862400,0.000000,28.140000,3227.000000,6.645100,0.875000,12.200000,0.000000,9.765110,691.000000
50%,100288.500000,6.000000,12.598435,1.090000,34.860000,4243.000000,7.988400,4.300000,44.300000,0.000000,61.010000,1084.000000
75%,100459.250000,8.000000,23.040000,156.060000,41.230000,5887.093750,9.436550,10.800000,190.500000,0.000000,204.392500,2328.000000
max,100617.000000,8.000000,59.040000,2555.520000,592.890000,52767.000000,24.600000,511.000000,4151.700000,1.000000,2068.110000,14610.000000


In [ ]:
print("Min date", df_2["DATE"].min(), ", Max Date: ", df_2["DATE"].max())

Min date 1/1/15 , Max Date:  9/9/16


In [ ]:
# concatenating the two df's

df = pd.concat([df_1, df_2])

In [ ]:

df.shape

(307751, 16)

In [ ]:
df.head()

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
0,100001,12/2/14,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880
1,100001,12/3/14,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881
2,100001,12/4/14,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882
3,100001,12/5/14,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883
4,100001,12/6/14,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884


ID — ID field that represents a specific machine.

DATE — The date of the observation.

REGION_CLUSTER — a field that represents the region in which the machine resides.

MAINTENANCE_VENDOR — a field that represents the company that provides maintenance and service to the machine.

MANUFACTURER — the company that manufactured the equipment in question.

WELL_GROUP — a field representing the type of machine.

EQUIPMENT_AGE — Age of the machine, in days.

S15 — A Sensor Value.

S17 — A Sensor Value.

S13 — A Sensor Value.

S16 — A Sensor Value.

S19 — A Sensor Value.

S18 — A Sensor Value.

S8 — A Sensor Value.

EQUIPMENT_FAILURE — A ‘1’ means that the equipment failed. A ‘0’ means the equipment did not fail.

Our first goal in this exercise is to build a model that predicts equipment failure. In other words, we will use the other variables in the data frame to predict EQUIPMENT_FAILURE

In [ ]:
df.describe()

,ID,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
count,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000
mean,100310.826603,4.543943,14.585192,80.265541,35.018249,4675.848252,7.972097,9.069123,137.963064,0.001368,144.665715,2524.192399
std,177.574390,2.284121,8.817056,85.804273,14.446585,2521.074632,2.321949,16.898887,238.890128,0.036961,240.773926,3158.930976
min,100001.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-16.490000,0.000000
25%,100161.000000,3.000000,7.694100,0.000000,28.200000,3209.000000,6.621500,0.900000,11.798276,0.000000,9.250000,721.000000
50%,100311.000000,5.000000,11.661600,31.680000,34.940000,4237.047619,8.004000,4.200000,38.200000,0.000000,53.080000,1113.000000
75%,100467.000000,6.000000,22.560000,160.080000,41.610000,5743.000000,9.460000,10.600000,150.900000,0.000000,165.092608,2784.000000
max,100617.000000,8.000000,59.040000,2555.520000,592.890000,52767.000000,24.600000,511.000000,4151.700000,1.000000,2068.110000,15170.000000


There are 421 machines

In [ ]:
print("Shape of the machine ID data: ", pd.DataFrame(df.groupby(["ID"]).agg(["count"])).shape)

Shape of the machine ID data:  (421, 15)


There are  unique dates

In [ ]:
print("Shape of the dates data: ", pd.DataFrame(df.groupby(["DATE"]).agg(["count"])).shape)

Shape of the dates data:  (731, 15)


We should have the same number of rows as 421 machines X 731 unique date entries

In [ ]:
421 * 731 == df.shape[0]

True

Therefore there shouldn't be any duplicates in the dataset..

In [ ]:
df.duplicated().sum(axis=0)

0

What about null values?

In [ ]:
df.isnull().sum(axis=0)

ID                    0
DATE                  0
REGION_CLUSTER        0
MAINTENANCE_VENDOR    0
MANUFACTURER          0
WELL_GROUP            0
S15                   0
S17                   0
S13                   0
S5                    0
S16                   0
S19                   0
S18                   0
EQUIPMENT_FAILURE     0
S8                    0
AGE_OF_EQUIPMENT      0
dtype: int64

Let's look at the distribution of the "EQUIPMENT_FAILURE" data... it seems that we do not have enough failure data (, which makes our dataset quite imbalanced.

which is quite common in reality (who wants to break their machines anyway?)

In [ ]:
failure = pd.DataFrame(df.groupby(["EQUIPMENT_FAILURE"])["ID"].agg(["count"]));
failure["%"] = round(failure["count"]/failure["count"].sum()*100, 2);
failure

,count,%
EQUIPMENT_FAILURE,,
0,307330,99.86
1,421,0.14


In [ ]:
df.corr(method="pearson")

,ID,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
ID,1.000000e+00,-3.744075e-02,-0.035295,0.026763,-0.037786,0.009999,0.053049,-0.040664,0.036977,2.959871e-18,0.037107,0.021574
WELL_GROUP,-3.744075e-02,1.000000e+00,0.049638,-0.065829,-0.069329,0.007193,0.014661,0.067174,0.091050,7.048348e-17,0.089935,-0.059788
S15,-3.529531e-02,4.963763e-02,1.000000,-0.610341,0.071915,0.261755,-0.113802,0.175694,0.380411,-6.036352e-02,0.330548,0.024427
S17,2.676294e-02,-6.582945e-02,-0.610341,1.000000,0.117863,-0.150205,0.167563,-0.153561,-0.409661,-3.429070e-02,-0.334659,-0.019681
S13,-3.778625e-02,-6.932946e-02,0.071915,0.117863,1.000000,0.422768,0.087766,-0.039499,-0.083504,-8.617761e-03,-0.048435,0.033191
S5,9.999313e-03,7.193471e-03,0.261755,-0.150205,0.422768,1.000000,-0.109097,0.269741,0.535484,-7.189979e-03,0.562171,0.034668
S16,5.304853e-02,1.466090e-02,-0.113802,0.167563,0.087766,-0.109097,1.000000,-0.126645,-0.044546,-6.138895e-03,-0.022518,0.018336
S19,-4.066430e-02,6.717403e-02,0.175694,-0.153561,-0.039499,0.269741,-0.126645,1.000000,0.230667,-6.087474e-03,0.186710,-0.026847
S18,3.697739e-02,9.105041e-02,0.380411,-0.409661,-0.083504,0.535484,-0.044546,0.230667,1.000000,9.765002e-03,0.856692,-0.027915
EQUIPMENT_FAILURE,2.959871e-18,7.048348e-17,-0.060364,-0.034291,-0.008618,-0.007190,-0.006139,-0.006087,0.009765,1.000000e+00,0.006517,0.000473


In [ ]:
# def scatter(df, x, cols, figsize=(20,12)):
#   ''' to create a multiple scatter plots filtered for each feature.
#   Input:
#     - df: name of the dataframe (dataframe)
#     - x: column name of the dependent variable (str)
#     - cols: names of the columns to be plotted against dependent variable (list)
#     - figsize: dimension of the figure (tuple, default: (20,12))

#   Output: 2-column scatter plots for each of the columns specified for a dataframe filtered to the well against date
#   ''' 
#   # setting up the subplot
#   n_plots = len(cols) + 1
#   plt.figure(figsize=figsize)

#   for i, col in enumerate(cols):
#       data = df
#       plt.subplot(n_plots//2, 2, i+1)
#       plt.scatter(y = data[col], x = data[x])
#       plt.title("{} vs {}".format(col)) 
#       plt.xlim([data[y].min(), data[y].max()])
#   plt.tight_layout()

In [ ]:
df.columns

Index(['ID', 'DATE', 'REGION_CLUSTER', 'MAINTENANCE_VENDOR', 'MANUFACTURER',
       'WELL_GROUP', 'S15', 'S17', 'S13', 'S5', 'S16', 'S19', 'S18',
       'EQUIPMENT_FAILURE', 'S8', 'AGE_OF_EQUIPMENT'],
      dtype='object')

In [ ]:
df.head()

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
0,100001,12/2/14,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880
1,100001,12/3/14,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881
2,100001,12/4/14,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882
3,100001,12/5/14,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883
4,100001,12/6/14,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884


In [ ]:
df.REGION_CLUSTER.unique()

array(['G', 'C', 'H', 'E', 'A', 'B', 'D', 'F'], dtype=object)

In [ ]:
df.MANUFACTURER.unique()

array(['Y', 'T', 'X', 'S', 'Q', 'Z', 'V', 'W', 'U', 'R'], dtype=object)

In [ ]:
df.MAINTENANCE_VENDOR.unique()

array(['O', 'L', 'M', 'K', 'J', 'I', 'N', 'P'], dtype=object)

In [ ]:
df["DATE"] = pd.to_datetime(df["DATE"])

In [ ]:
# sorting it by ID first and then DATE
df = df.sort_values(by=["ID", "DATE"], ascending=[True,True])

In [ ]:
df.head(200)

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT,flipper
0,100001,2014-12-02,G,O,Y,1,11.088000,145.223448,39.340000,3501.000000,8.426869,1.9,24.610345,0,0.000000,880,1
1,100001,2014-12-03,G,O,Y,1,8.877943,187.573214,39.200000,3489.000000,6.483714,1.9,24.671429,0,0.000000,881,0
2,100001,2014-12-04,G,O,Y,1,8.676444,148.363704,38.870000,3459.000000,6.159659,2.0,24.733333,0,0.000000,882,0
3,100001,2014-12-05,G,O,Y,1,9.988338,133.660000,39.470000,3513.000000,9.320308,2.0,24.773077,0,0.000000,883,0
4,100001,2014-12-06,G,O,Y,1,8.475264,197.181600,40.330000,3589.000000,8.022960,1.5,24.808000,0,0.000000,884,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,100001,2015-06-15,G,O,Y,1,18.240000,0.000000,40.940417,3643.708333,11.376000,3.9,58.000000,0,57.840833,1075,0
196,100001,2015-06-16,G,O,Y,1,26.400000,0.000000,40.892609,3639.434783,10.333200,4.5,58.000000,0,58.521739,1076,0
197,100001,2015-06-17,G,O,Y,1,21.120000,0.000000,41.550000,3698.000000,7.584000,4.5,56.700000,0,51.780000,1077,0
198,100001,2015-06-18,G,O,Y,1,22.320000,0.000000,41.300000,3676.000000,9.195600,4.5,56.800000,0,44.430000,1078,0


In [ ]:
df[df.flipper==1]

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT,flipper
0,100001,2014-12-02,G,O,Y,1,11.088000,145.223448,39.34,3501.000000,8.426869,1.90,24.610345,0,0.000000,880,1
0,100002,2014-12-02,D,L,R,6,23.520000,0.000000,34.15,4064.000000,7.842741,31.10,68.331034,0,0.000000,820,1
731,100003,2014-12-02,C,L,T,1,11.128621,191.855172,35.04,3119.000000,9.771203,1.40,7.275862,0,0.000000,400,1
731,100006,2014-12-02,B,J,S,7,24.240000,0.000000,53.11,3718.000000,11.595517,0.30,293.837931,0,0.000000,980,1
1462,100007,2014-12-02,B,N,W,8,18.720000,0.000000,40.22,4786.000000,2.656521,24.00,53.193103,0,0.000000,1380,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149124,100612,2014-12-02,H,O,V,4,20.203614,5.775862,27.46,8376.000000,10.257793,48.00,274.324138,0,305.140000,360,1
154972,100613,2014-12-02,D,O,Y,5,18.384483,99.127586,15.39,5616.000000,6.752200,21.02,108.841379,0,472.380000,440,1
155703,100614,2014-12-02,A,K,T,7,0.000000,0.000000,35.61,4238.000000,8.584000,7.10,0.000000,0,0.000000,740,1
156434,100616,2014-12-02,A,O,S,8,14.289683,126.897931,38.56,9872.000000,5.757983,0.00,25.075862,0,106.870000,560,1


We want to create a "flipper" to indicate the row at which is the end of the machine 

In [ ]:
df["flipper"] = np.where((df["ID"] != df["ID"].shift(1)), 1, 0); df.head()

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT,flipper
0,100001,2014-12-02,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880,1
1,100001,2014-12-03,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881,0
2,100001,2014-12-04,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882,0
3,100001,2014-12-05,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883,0
4,100001,2014-12-06,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884,0


In [ ]:
# save it so that we don't have to rerun all the above
df.to_pickle(path / "df_with_flipper")

In [ ]:
# load saved pickle data
data = pd.read_pickle(path / "df_with_flipper")

We will be using moving average to get the average value over the last "window" days. 

In [ ]:
# define the feature window to aggregate our sensor values
feature_window = 21

In [ ]:
startdate_df = data[data["flipper"] == 1][["DATE", "ID"]]
startdate_df

,DATE,ID
0,2014-12-02,100001
731,2014-12-02,100002
1462,2014-12-02,100003
2193,2014-12-02,100006
2924,2014-12-02,100007
...,...,...
304096,2014-12-02,100612
304827,2014-12-02,100613
305558,2014-12-02,100614
306289,2014-12-02,100616


In [ ]:
startdate = startdate_df.rename(columns = {"DATE" : "START_DATE"})

In [ ]:
startdate

,START_DATE,ID
0,2014-12-02,100001
0,2014-12-02,100002
731,2014-12-02,100003
731,2014-12-02,100006
1462,2014-12-02,100007
...,...,...
149124,2014-12-02,100612
154972,2014-12-02,100613
155703,2014-12-02,100614
156434,2014-12-02,100616


In [ ]:
data = pd.merge(data, startdate, on="ID", how="left")

In [ ]:
data.head(148)

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT,flipper,START_DATE
0,100001,2014-12-02,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.00,880,1,2014-12-02
1,100001,2014-12-03,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.00,881,0,2014-12-02
2,100001,2014-12-04,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.00,882,0,2014-12-02
3,100001,2014-12-05,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.00,883,0,2014-12-02
4,100001,2014-12-06,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.00,884,0,2014-12-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307746,100617,2016-11-27,C,P,S,7,29.520000,0.000000,48.99,3282.0,8.960800,0.2,22.500000,0,15.00,806,0,2014-12-02
307747,100617,2016-11-28,C,P,S,7,24.240000,0.000000,45.73,3064.0,11.201000,0.2,13.200000,0,11.10,807,0,2014-12-02
307748,100617,2016-11-29,C,P,S,7,28.560000,0.000000,46.40,3109.0,10.400400,0.2,16.100000,0,11.12,808,0,2014-12-02
307749,100617,2016-11-30,C,P,S,7,23.280000,0.000000,46.10,3089.0,12.052800,0.2,12.000000,0,11.28,809,0,2014-12-02


In [ ]:
data["T"] = data["DATE"] - data["START_DATE"]
data["TIME_SINCE_START"] =  ( data["DATE"] - data["START_DATE"] ) / np.timedelta64(1, "D")
data["within_window"] = np.where( (data["TIME_SINCE_START"] < feature_window), 1, 0)
data.drop(columns=["T"], inplace=True)
data.head(25)

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT,flipper,START_DATE,TIME_SINCE_START,within_window,S15_mean,S17_mean,S13_mean,S5_mean,S16_mean,S19_mean,S18_mean,S8_mean,S15_median,S15_max,S15_min,S17_median,S17_max,S17_min,S13_median,S13_max,S13_min,S5_median,S5_max,S5_min,S16_median,S16_max,S16_min,S19_median,S19_max,S19_min,S18_median,S18_max,S18_min,S8_median,S8_max,S8_min,S15_change,S17_change,S13_change,S5_change,S16_change,S19_change,S18_change,S8_change
0,100001,2014-12-02,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880,1,2014-12-02,0.0,1,11.088000,145.223448,39.340000,3501.000000,8.426869,1.900000,24.610345,0.0,11.088000,11.088000,11.088000,145.223448,145.223448,145.223448,39.340,39.34,39.34,3501.0,3501.0,3501.0,8.426869,8.426869,8.426869,1.90,1.9,1.9,24.610345,24.610345,24.610345,0.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
1,100001,2014-12-03,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881,0,2014-12-02,1.0,1,9.982971,166.398331,39.270000,3495.000000,7.455291,1.900000,24.640887,0.0,9.982971,11.088000,8.877943,166.398331,187.573214,145.223448,39.270,39.34,39.20,3495.0,3501.0,3489.0,7.455291,8.426869,6.483714,1.90,1.9,1.9,24.640887,24.671429,24.610345,0.0,0.0,0.0,0.889309,1.127254,0.998217,0.998283,0.869679,1.000000,1.001239,0.0
2,100001,2014-12-04,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882,0,2014-12-02,2.0,1,9.547462,160.386789,39.136667,3483.000000,7.023414,1.933333,24.671702,0.0,8.877943,11.088000,8.676444,148.363704,187.573214,145.223448,39.200,39.34,38.87,3489.0,3501.0,3459.0,6.483714,8.426869,6.159659,1.90,2.0,1.9,24.671429,24.733333,24.610345,0.0,0.0,0.0,0.908770,0.925037,0.993186,0.993109,0.877018,1.034483,1.002498,0.0
3,100001,2014-12-05,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883,0,2014-12-02,3.0,1,9.657681,153.705091,39.220000,3490.500000,7.597638,1.950000,24.697046,0.0,9.433141,11.088000,8.676444,146.793576,187.573214,133.660000,39.270,39.47,38.87,3495.0,3513.0,3459.0,7.455291,9.320308,6.159659,1.95,2.0,1.9,24.702381,24.773077,24.610345,0.0,0.0,0.0,1.034238,0.869587,1.006374,1.006446,1.226738,1.025641,1.003079,0.0
4,100001,2014-12-06,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884,0,2014-12-02,4.0,1,9.421198,162.400393,39.442000,3510.200000,7.682702,1.860000,24.719237,0.0,8.877943,11.088000,8.475264,148.363704,197.181600,133.660000,39.340,40.33,38.87,3501.0,3589.0,3459.0,8.022960,9.320308,6.159659,1.90,2.0,1.5,24.733333,24.808000,24.610345,0.0,0.0,0.0,0.899595,1.214169,1.022514,1.022449,1.044289,0.806452,1.003591,0.0
5,100001,2014-12-07,G,O,Y,1,7.971100,164.545833,38.74,3448.0,6.645075,1.5,24.845833,0,0.0,885,0,2014-12-02,5.0,1,9.179515,162.757966,39.325000,3499.833333,7.509764,1.800000,24.740336,0.0,8.777193,11.088000,7.971100,156.454769,197.181600,133.660000,39.270,40.33,38.74,3495.0,3589.0,3448.0,7.334018,9.320308,6.159659,1.90,2.0,1.5,24.753205,24.845833,24.610345,0.0,0.0,0.0,0.868357,1.010985,0.985124,0.985190,0.884858,0.833333,1.004264,0.0
6,100001,2014-12-08,G,O,Y,1,11.009948,175.852174,38.84,3457.0,6.401404,1.5,24.886957,0,0.0,886,0,2014-12-02,6.0,1,9.441005,164.628568,39.255714,3493.714286,7.351427,1.757143,24.761282,0.0,8.877943,11.088000,7.971100,164.545833,197.181600,133.660000,39.200,40.33,38.74,3489.0,3589.0,3448.0,6.645075,9.320308,6.159659,1.90,2.0,1.5,24.773077,24.886957,24.610345,0.0,0.0,0.0,1.166184,1.068175,0.989410,0.989491,0.870770,0.853659,1.005075,0.0
7,100001,2014-12-09,G,O,Y,1,8.283491,136.767273,39.58,3523.0,8.507864,2.9,24.904545,0,0.0,887,0,2014-12-02,7.0,1,9.296316,161.145906,39.296250,3497.375000,7.495982,1.900000,24.779190,0.0,8.777193,11.088000,7.971100,156.454769,197.181600,133.660000,39.270,40.33,38.74,3495.0,3589.0,3448.0,7.334018,9.320308,6.159659,1.90,2.9,1.5,24.790539,24.904545,24.610345,0.0,0.0,0.0,0.891051,0.848717,

In [ ]:
data.columns

Index(['ID', 'DATE', 'REGION_CLUSTER', 'MAINTENANCE_VENDOR', 'MANUFACTURER',
       'WELL_GROUP', 'S15', 'S17', 'S13', 'S5', 'S16', 'S19', 'S18',
       'EQUIPMENT_FAILURE', 'S8', 'AGE_OF_EQUIPMENT', 'flipper', 'START_DATE',
       'T', 'TIME_SINCE_START', 'not_yet'],
      dtype='object')

In [ ]:
sensors_name = ['S15', 'S17', 'S13', 'S5', 'S16', 'S19', 'S18',"S8"]

# create a rolling statistics for the sensor values
for sensor in sensors_name:
  data[sensor + str("_mean")] = np.where((data["within_window"] == 1), ( data[sensor].rolling(min_periods=1, window=feature_window).mean() ), data[sensor])
  data[sensor + str("_median")] = np.where((data["within_window"] == 1), ( data[sensor].rolling(min_periods=1, window=feature_window).median() ), data[sensor])
  data[sensor + str("_max")] = np.where((data["within_window"] == 1), ( data[sensor].rolling(min_periods=1, window=feature_window).max() ), data[sensor])
  data[sensor + str("_min")] = np.where((data["within_window"] == 1), ( data[sensor].rolling(min_periods=1, window=feature_window).min() ), data[sensor])

In [ ]:
data.head(25)

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT,flipper,START_DATE,T,TIME_SINCE_START,within_window,S15_mean,S17_mean,S13_mean,S5_mean,S16_mean,S19_mean,S18_mean,S8_mean,S15_median,S15_max,S15_min,S17_median,S17_max,S17_min,S13_median,S13_max,S13_min,S5_median,S5_max,S5_min,S16_median,S16_max,S16_min,S19_median,S19_max,S19_min,S18_median,S18_max,S18_min,S8_median,S8_max,S8_min
0,100001,2014-12-02,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880,1,2014-12-02,0 days,0.0,1,11.088000,145.223448,39.340000,3501.000000,8.426869,1.900000,24.610345,0.0,11.088000,11.088000,11.088000,145.223448,145.223448,145.223448,39.340,39.34,39.34,3501.0,3501.0,3501.0,8.426869,8.426869,8.426869,1.90,1.9,1.9,24.610345,24.610345,24.610345,0.0,0.0,0.0
1,100001,2014-12-03,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881,0,2014-12-02,1 days,1.0,1,9.982971,166.398331,39.270000,3495.000000,7.455291,1.900000,24.640887,0.0,9.982971,11.088000,8.877943,166.398331,187.573214,145.223448,39.270,39.34,39.20,3495.0,3501.0,3489.0,7.455291,8.426869,6.483714,1.90,1.9,1.9,24.640887,24.671429,24.610345,0.0,0.0,0.0
2,100001,2014-12-04,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882,0,2014-12-02,2 days,2.0,1,9.547462,160.386789,39.136667,3483.000000,7.023414,1.933333,24.671702,0.0,8.877943,11.088000,8.676444,148.363704,187.573214,145.223448,39.200,39.34,38.87,3489.0,3501.0,3459.0,6.483714,8.426869,6.159659,1.90,2.0,1.9,24.671429,24.733333,24.610345,0.0,0.0,0.0
3,100001,2014-12-05,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883,0,2014-12-02,3 days,3.0,1,9.657681,153.705091,39.220000,3490.500000,7.597638,1.950000,24.697046,0.0,9.433141,11.088000,8.676444,146.793576,187.573214,133.660000,39.270,39.47,38.87,3495.0,3513.0,3459.0,7.455291,9.320308,6.159659,1.95,2.0,1.9,24.702381,24.773077,24.610345,0.0,0.0,0.0
4,100001,2014-12-06,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884,0,2014-12-02,4 days,4.0,1,9.421198,162.400393,39.442000,3510.200000,7.682702,1.860000,24.719237,0.0,8.877943,11.088000,8.475264,148.363704,197.181600,133.660000,39.340,40.33,38.87,3501.0,3589.0,3459.0,8.022960,9.320308,6.159659,1.90,2.0,1.5,24.733333,24.808000,24.610345,0.0,0.0,0.0
5,100001,2014-12-07,G,O,Y,1,7.971100,164.545833,38.74,3448.0,6.645075,1.5,24.845833,0,0.0,885,0,2014-12-02,5 days,5.0,1,9.179515,162.757966,39.325000,3499.833333,7.509764,1.800000,24.740336,0.0,8.777193,11.088000,7.971100,156.454769,197.181600,133.660000,39.270,40.33,38.74,3495.0,3589.0,3448.0,7.334018,9.320308,6.159659,1.90,2.0,1.5,24.753205,24.845833,24.610345,0.0,0.0,0.0
6,100001,2014-12-08,G,O,Y,1,11.009948,175.852174,38.84,3457.0,6.401404,1.5,24.886957,0,0.0,886,0,2014-12-02,6 days,6.0,1,9.441005,164.628568,39.255714,3493.714286,7.351427,1.757143,24.761282,0.0,8.877943,11.088000,7.971100,164.545833,197.181600,133.660000,39.200,40.33,38.74,3489.0,3589.0,3448.0,6.645075,9.320308,6.159659,1.90,2.0,1.5,24.773077,24.886957,24.610345,0.0,0.0,0.0
7,100001,2014-12-09,G,O,Y,1,8.283491,136.767273,39.58,3523.0,8.507864,2.9,24.904545,0,0.0,887,0,2014-12-02,7 days,7.0,1,9.296316,161.145906,39.296250,3497.375000,7.495982,1.900000,24.779190,0.0,8.777193,11.088000,7.971100,156.454769,197.181600,133.660000,39.270,40.33,38.74,3495.0,3589.0,3448.0,7.334018,9.320308,6.159659,1.90,2.9,1.5,24.790539,24.904545,24.610345,0.0,0.0,0.0
8,100001,2014-12-10,G,O,Y,1,7.577143,182.293333,39.78,3540.0,9.884905,2.9,24.933333,0,0.0,888,0,2014-12-02,8 days,8.0,1,9.105297,163.495620,39.350000,3502.111111,7.761418,2.011111,24.796317,0.0,8.676444,11.088000,7.577143,164.545833,197.181600,133.660000,39.340,40.33,38.74,3501.0,3589.0,3448.0,8.022960,9.884905,6.159659,1.90,2.9,1.5,24.808000,24.933333,24.610345,0.0,0.0,0.0
9,100001,2014-12-11,G,O,Y,1,8.487360,203.375000,39.54,3519.0,9.883830,3.3,24.945000,0,0.0,889,0,2014-12-02,9 days,9.0,1,9.043503,167.48355

Another useful thing to do is to detect sudden spikes in sensor values to show deviation from the average values..

In [ ]:
sensors_name = ['S15', 'S17', 'S13', 'S5', 'S16', 'S19', 'S18',"S8"]

# calculate fractional deviations from mean:
for sensor in sensors_name:
  data[sensor + str("_change")] = np.where( (data[sensor + str("_mean")] == 0 ), 0, ( data[sensor] / data[sensor + str("_mean")] ) )   # if mean = 0, then change is zero, otherwise sensor_value / sensor_mean_value

data.head(25)

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT,flipper,START_DATE,T,TIME_SINCE_START,within_window,S15_mean,S17_mean,S13_mean,S5_mean,S16_mean,S19_mean,S18_mean,S8_mean,S15_median,S15_max,S15_min,S17_median,S17_max,S17_min,S13_median,S13_max,S13_min,S5_median,S5_max,S5_min,S16_median,S16_max,S16_min,S19_median,S19_max,S19_min,S18_median,S18_max,S18_min,S8_median,S8_max,S8_min,S15_change,S17_change,S13_change,S5_change,S16_change,S19_change,S18_change,S8_change
0,100001,2014-12-02,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880,1,2014-12-02,0 days,0.0,1,11.088000,145.223448,39.340000,3501.000000,8.426869,1.900000,24.610345,0.0,11.088000,11.088000,11.088000,145.223448,145.223448,145.223448,39.340,39.34,39.34,3501.0,3501.0,3501.0,8.426869,8.426869,8.426869,1.90,1.9,1.9,24.610345,24.610345,24.610345,0.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
1,100001,2014-12-03,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881,0,2014-12-02,1 days,1.0,1,9.982971,166.398331,39.270000,3495.000000,7.455291,1.900000,24.640887,0.0,9.982971,11.088000,8.877943,166.398331,187.573214,145.223448,39.270,39.34,39.20,3495.0,3501.0,3489.0,7.455291,8.426869,6.483714,1.90,1.9,1.9,24.640887,24.671429,24.610345,0.0,0.0,0.0,0.889309,1.127254,0.998217,0.998283,0.869679,1.000000,1.001239,0.0
2,100001,2014-12-04,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882,0,2014-12-02,2 days,2.0,1,9.547462,160.386789,39.136667,3483.000000,7.023414,1.933333,24.671702,0.0,8.877943,11.088000,8.676444,148.363704,187.573214,145.223448,39.200,39.34,38.87,3489.0,3501.0,3459.0,6.483714,8.426869,6.159659,1.90,2.0,1.9,24.671429,24.733333,24.610345,0.0,0.0,0.0,0.908770,0.925037,0.993186,0.993109,0.877018,1.034483,1.002498,0.0
3,100001,2014-12-05,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883,0,2014-12-02,3 days,3.0,1,9.657681,153.705091,39.220000,3490.500000,7.597638,1.950000,24.697046,0.0,9.433141,11.088000,8.676444,146.793576,187.573214,133.660000,39.270,39.47,38.87,3495.0,3513.0,3459.0,7.455291,9.320308,6.159659,1.95,2.0,1.9,24.702381,24.773077,24.610345,0.0,0.0,0.0,1.034238,0.869587,1.006374,1.006446,1.226738,1.025641,1.003079,0.0
4,100001,2014-12-06,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884,0,2014-12-02,4 days,4.0,1,9.421198,162.400393,39.442000,3510.200000,7.682702,1.860000,24.719237,0.0,8.877943,11.088000,8.475264,148.363704,197.181600,133.660000,39.340,40.33,38.87,3501.0,3589.0,3459.0,8.022960,9.320308,6.159659,1.90,2.0,1.5,24.733333,24.808000,24.610345,0.0,0.0,0.0,0.899595,1.214169,1.022514,1.022449,1.044289,0.806452,1.003591,0.0
5,100001,2014-12-07,G,O,Y,1,7.971100,164.545833,38.74,3448.0,6.645075,1.5,24.845833,0,0.0,885,0,2014-12-02,5 days,5.0,1,9.179515,162.757966,39.325000,3499.833333,7.509764,1.800000,24.740336,0.0,8.777193,11.088000,7.971100,156.454769,197.181600,133.660000,39.270,40.33,38.74,3495.0,3589.0,3448.0,7.334018,9.320308,6.159659,1.90,2.0,1.5,24.753205,24.845833,24.610345,0.0,0.0,0.0,0.868357,1.010985,0.985124,0.985190,0.884858,0.833333,1.004264,0.0
6,100001,2014-12-08,G,O,Y,1,11.009948,175.852174,38.84,3457.0,6.401404,1.5,24.886957,0,0.0,886,0,2014-12-02,6 days,6.0,1,9.441005,164.628568,39.255714,3493.714286,7.351427,1.757143,24.761282,0.0,8.877943,11.088000,7.971100,164.545833,197.181600,133.660000,39.200,40.33,38.74,3489.0,3589.0,3448.0,6.645075,9.320308,6.159659,1.90,2.0,1.5,24.773077,24.886957,24.610345,0.0,0.0,0.0,1.166184,1.068175,0.989410,0.989491,0.870770,0.853659,1.005075,0.0
7,100001,2014-12-09,G,O,Y,1,8.283491,136.767273,39.58,3523.0,8.507864,2.9,24.904545,0,0.0,887,0,2014-12-02,7 days,7.0,1,9.296316,161.145906,39.296250,3497.375000,7.495982,1.900000,24.779190,0.0,8.777193,11.088000,7.971100,156.454769,197.181600,133.660000,39.270,40.33,38.74,3495.0,3589.0,3448.0,7.334018,9.320308,6.159659,1.90,2.9,1.5,24

In [ ]:
#savepoint! 

data.to_pickle(path / "data_with_rolling_spikes")

In [ ]:
# load saved pickle

data = pd.read_pickle(path / "data_with_rolling_spikes")

# Dealing with Uneven Data Distributions

Remember that our failure data consists only about 0.14% of our whole dataset, which is extremely low and the ML algorithm will not be able to learn it very well.

This is very common, considering that machines are usually made to last and we don't typically run the machines to failure.

To ameliorate the issue of imbalance in the dataset, one of the ways is to augment the failure data into a window of time, in which case we will call it `target_window`. 

This `target_window` is the number of days prior to the failure time that we will consider to be "failed". For example, if we set `target_window` to 28 days, that means the machine was already "failed" 28 prior to the actual failure day.

In [ ]:
# set the target_window days

target_window = 28

In [ ]:
# create a dataframe with only the failure date for each ID 

data = data.sort_values(by=["ID", "DATE"], ascending=[True, True])

failure_df = data[data["EQUIPMENT_FAILURE"] == 1][["DATE", "ID"]];
failure_df = failure_df.rename(columns = {"DATE" : "FAILURE_DATE"})
failure_df

,FAILURE_DATE,ID
143,2015-04-24,100001
1060,2015-10-27,100002
2183,2016-11-22,100003
2754,2016-06-15,100006
2984,2015-01-31,100007
...,...,...
304742,2016-09-08,100612
305182,2015-11-22,100613
306127,2016-06-23,100614
306848,2016-06-13,100616


In [ ]:
# merge to the main dataframe

data = data.sort_values(by=["ID"], ascending=True)
failure_df = failure_df.sort_values(by=["ID"], ascending=True)

data = pd.merge(data, failure_df, how="left", on = "ID")

In [ ]:
data.head()

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT,flipper,START_DATE,TIME_SINCE_START,within_window,S15_mean,S17_mean,S13_mean,S5_mean,S16_mean,S19_mean,S18_mean,S8_mean,S15_median,S15_max,S15_min,S17_median,S17_max,S17_min,S13_median,S13_max,S13_min,S5_median,S5_max,S5_min,S16_median,S16_max,S16_min,S19_median,S19_max,S19_min,S18_median,S18_max,S18_min,S8_median,S8_max,S8_min,S15_change,S17_change,S13_change,S5_change,S16_change,S19_change,S18_change,S8_change,FAILURE_DATE
0,100001,2014-12-02,G,O,Y,1,11.088,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.00,880,1,2014-12-02,0.0,1,11.088,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0.00,11.088,11.088,11.088,145.223448,145.223448,145.223448,39.34,39.34,39.34,3501.0,3501.0,3501.0,8.426869,8.426869,8.426869,1.9,1.9,1.9,24.610345,24.610345,24.610345,0.00,0.00,0.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2015-04-24
1,100001,2016-03-29,G,O,Y,1,18.960,0.000000,38.87,3459.0,10.047300,1.3,36.600000,0,34.37,1363,0,2014-12-02,483.0,0,18.960,0.000000,38.87,3459.0,10.047300,1.3,36.600000,34.37,18.960,18.960,18.960,0.000000,0.000000,0.000000,38.87,38.87,38.87,3459.0,3459.0,3459.0,10.047300,10.047300,10.047300,1.3,1.3,1.3,36.600000,36.600000,36.600000,34.37,34.37,34.37,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2015-04-24
2,100001,2016-03-30,G,O,Y,1,29.040,0.000000,37.36,3325.0,10.235100,1.4,36.000000,0,32.37,1364,0,2014-12-02,484.0,0,29.040,0.000000,37.36,3325.0,10.235100,1.4,36.000000,32.37,29.040,29.040,29.040,0.000000,0.000000,0.000000,37.36,37.36,37.36,3325.0,3325.0,3325.0,10.235100,10.235100,10.235100,1.4,1.4,1.4,36.000000,36.000000,36.000000,32.37,32.37,32.37,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2015-04-24
3,100001,2016-03-31,G,O,Y,1,18.000,0.000000,38.81,3454.0,8.544900,1.4,36.100000,0,34.44,1365,0,2014-12-02,485.0,0,18.000,0.000000,38.81,3454.0,8.544900,1.4,36.100000,34.44,18.000,18.000,18.000,0.000000,0.000000,0.000000,38.81,38.81,38.81,3454.0,3454.0,3454.0,8.544900,8.544900,8.544900,1.4,1.4,1.4,36.100000,36.100000,36.100000,34.44,34.44,34.44,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2015-04-24
4,100001,2016-04-01,G,O,Y,1,26.160,0.000000,39.47,3513.0,10.986300,1.4,36.300000,0,33.26,1366,0,2014-12-02,486.0,0,26.160,0.000000,39.47,3513.0,10.986300,1.4,36.300000,33.26,26.160,26.160,26.160,0.000000,0.000000,0.000000,39.47,39.47,39.47,3513.0,3513.0,3513.0,10.986300,10.986300,10.986300,1.4,1.4,1.4,36.300000,36.300000,36.300000,33.26,33.26,33.26,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2015-04-24


In [ ]:
data["FAILURE_DATE"] = pd.to_datetime(data["FAILURE_DATE"])
data["DATE"] = pd.to_datetime(data["DATE"])

data["TIME_TO_FAILURE"] = ( data["FAILURE_DATE"]  - data["DATE"] ) / np.timedelta64(1, 'D')
data = data.sort_values(by=['ID', 'DATE'], ascending=[True, True])
data.reset_index(inplace=True)

In [ ]:
data.head(148)

,index,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT,flipper,START_DATE,TIME_SINCE_START,within_window,S15_mean,S17_mean,S13_mean,S5_mean,S16_mean,S19_mean,S18_mean,S8_mean,S15_median,S15_max,S15_min,S17_median,S17_max,S17_min,S13_median,S13_max,S13_min,S5_median,S5_max,S5_min,S16_median,S16_max,S16_min,S19_median,S19_max,S19_min,S18_median,S18_max,S18_min,S8_median,S8_max,S8_min,S15_change,S17_change,S13_change,S5_change,S16_change,S19_change,S18_change,S8_change,FAILURE_DATE,TIME_TO_FAILURE
0,0,100001,2014-12-02,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880,1,2014-12-02,0.0,1,11.088000,145.223448,39.340000,3501.0,8.426869,1.900000,24.610345,0.0,11.088000,11.088,11.088000,145.223448,145.223448,145.223448,39.34,39.34,39.34,3501.0,3501.0,3501.0,8.426869,8.426869,8.426869,1.90,1.9,1.9,24.610345,24.610345,24.610345,0.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,2015-04-24,143.0
1,549,100001,2014-12-03,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881,0,2014-12-02,1.0,1,9.982971,166.398331,39.270000,3495.0,7.455291,1.900000,24.640887,0.0,9.982971,11.088,8.877943,166.398331,187.573214,145.223448,39.27,39.34,39.20,3495.0,3501.0,3489.0,7.455291,8.426869,6.483714,1.90,1.9,1.9,24.640887,24.671429,24.610345,0.0,0.0,0.0,0.889309,1.127254,0.998217,0.998283,0.869679,1.000000,1.001239,0.0,2015-04-24,142.0
2,483,100001,2014-12-04,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882,0,2014-12-02,2.0,1,9.547462,160.386789,39.136667,3483.0,7.023414,1.933333,24.671702,0.0,8.877943,11.088,8.676444,148.363704,187.573214,145.223448,39.20,39.34,38.87,3489.0,3501.0,3459.0,6.483714,8.426869,6.159659,1.90,2.0,1.9,24.671429,24.733333,24.610345,0.0,0.0,0.0,0.908770,0.925037,0.993186,0.993109,0.877018,1.034483,1.002498,0.0,2015-04-24,141.0
3,484,100001,2014-12-05,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883,0,2014-12-02,3.0,1,9.657681,153.705091,39.220000,3490.5,7.597638,1.950000,24.697046,0.0,9.433141,11.088,8.676444,146.793576,187.573214,133.660000,39.27,39.47,38.87,3495.0,3513.0,3459.0,7.455291,9.320308,6.159659,1.95,2.0,1.9,24.702381,24.773077,24.610345,0.0,0.0,0.0,1.034238,0.869587,1.006374,1.006446,1.226738,1.025641,1.003079,0.0,2015-04-24,140.0
4,485,100001,2014-12-06,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884,0,2014-12-02,4.0,1,9.421198,162.400393,39.442000,3510.2,7.682702,1.860000,24.719237,0.0,8.877943,11.088,8.475264,148.363704,197.181600,133.660000,39.34,40.33,38.87,3501.0,3589.0,3459.0,8.022960,9.320308,6.159659,1.90,2.0,1.5,24.733333,24.808000,24.610345,0.0,0.0,0.0,0.899595,1.214169,1.022514,1.022449,1.044289,0.806452,1.003591,0.0,2015-04-24,139.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,440,100001,2015-04-24,G,O,Y,1,0.000000,0.000000,0.00,0.0,8.892400,0.9,170.400000,1,0.0,1023,0,2014-12-02,143.0,0,0.000000,0.000000,0.000000,0.0,8.892400,0.900000,170.400000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.0,8.892400,8.892400,8.892400,0.90,0.9,0.9,170.400000,170.400000,170.400000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,2015-04-24,0.0
144,441,100001,2015-04-25,G,O,Y,1,0.000000,0.000000,0.00,0.0,9.649200,0.9,170.400000,0,0.0,1024,0,2014-12-02,144.0,0,0.000000,0.000000,0.000000,0.0,9.649200,0.900000,170.400000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.0,9.649200,9.649200,9.649200,0.90,0.9,0.9,170.400000,170.400000,170.400000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,2015-04-24,-1.0
145,442,100001,2015-04-26,G,O,Y,1,0.000000,0.000000,0.00,0.0,

Create a target variable `TARGET_FAILURE` that sets to 1 if `TIME_TO_FAILURE` < `target_window` (even when `EQUIPMENT_FAILURE` is still 0)


In [ ]:
# create the TARGET_FAILURE column

data["TARGET_FAILURE"] = np.where(( (data["TIME_TO_FAILURE"] < target_window) & (data["TIME_TO_FAILURE"] >= 0) ), 1, 0)

In [ ]:
data.head(148)

,index,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT,flipper,START_DATE,TIME_SINCE_START,within_window,S15_mean,S17_mean,S13_mean,S5_mean,S16_mean,S19_mean,S18_mean,S8_mean,S15_median,S15_max,S15_min,S17_median,S17_max,S17_min,S13_median,S13_max,S13_min,S5_median,S5_max,S5_min,S16_median,S16_max,S16_min,S19_median,S19_max,S19_min,S18_median,S18_max,S18_min,S8_median,S8_max,S8_min,S15_change,S17_change,S13_change,S5_change,S16_change,S19_change,S18_change,S8_change,FAILURE_DATE,TIME_TO_FAILURE,TARGET_FAILURE
0,0,100001,2014-12-02,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880,1,2014-12-02,0.0,1,11.088000,145.223448,39.340000,3501.0,8.426869,1.900000,24.610345,0.0,11.088000,11.088,11.088000,145.223448,145.223448,145.223448,39.34,39.34,39.34,3501.0,3501.0,3501.0,8.426869,8.426869,8.426869,1.90,1.9,1.9,24.610345,24.610345,24.610345,0.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,2015-04-24,143.0,0
1,549,100001,2014-12-03,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881,0,2014-12-02,1.0,1,9.982971,166.398331,39.270000,3495.0,7.455291,1.900000,24.640887,0.0,9.982971,11.088,8.877943,166.398331,187.573214,145.223448,39.27,39.34,39.20,3495.0,3501.0,3489.0,7.455291,8.426869,6.483714,1.90,1.9,1.9,24.640887,24.671429,24.610345,0.0,0.0,0.0,0.889309,1.127254,0.998217,0.998283,0.869679,1.000000,1.001239,0.0,2015-04-24,142.0,0
2,483,100001,2014-12-04,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882,0,2014-12-02,2.0,1,9.547462,160.386789,39.136667,3483.0,7.023414,1.933333,24.671702,0.0,8.877943,11.088,8.676444,148.363704,187.573214,145.223448,39.20,39.34,38.87,3489.0,3501.0,3459.0,6.483714,8.426869,6.159659,1.90,2.0,1.9,24.671429,24.733333,24.610345,0.0,0.0,0.0,0.908770,0.925037,0.993186,0.993109,0.877018,1.034483,1.002498,0.0,2015-04-24,141.0,0
3,484,100001,2014-12-05,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883,0,2014-12-02,3.0,1,9.657681,153.705091,39.220000,3490.5,7.597638,1.950000,24.697046,0.0,9.433141,11.088,8.676444,146.793576,187.573214,133.660000,39.27,39.47,38.87,3495.0,3513.0,3459.0,7.455291,9.320308,6.159659,1.95,2.0,1.9,24.702381,24.773077,24.610345,0.0,0.0,0.0,1.034238,0.869587,1.006374,1.006446,1.226738,1.025641,1.003079,0.0,2015-04-24,140.0,0
4,485,100001,2014-12-06,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884,0,2014-12-02,4.0,1,9.421198,162.400393,39.442000,3510.2,7.682702,1.860000,24.719237,0.0,8.877943,11.088,8.475264,148.363704,197.181600,133.660000,39.34,40.33,38.87,3501.0,3589.0,3459.0,8.022960,9.320308,6.159659,1.90,2.0,1.5,24.733333,24.808000,24.610345,0.0,0.0,0.0,0.899595,1.214169,1.022514,1.022449,1.044289,0.806452,1.003591,0.0,2015-04-24,139.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,440,100001,2015-04-24,G,O,Y,1,0.000000,0.000000,0.00,0.0,8.892400,0.9,170.400000,1,0.0,1023,0,2014-12-02,143.0,0,0.000000,0.000000,0.000000,0.0,8.892400,0.900000,170.400000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.0,8.892400,8.892400,8.892400,0.90,0.9,0.9,170.400000,170.400000,170.400000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,2015-04-24,0.0,1
144,441,100001,2015-04-25,G,O,Y,1,0.000000,0.000000,0.00,0.0,9.649200,0.9,170.400000,0,0.0,1024,0,2014-12-02,144.0,0,0.000000,0.000000,0.000000,0.0,9.649200,0.900000,170.400000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.0,9.649200,9.649200,9.649200,0.90,0.9,0.9,170.400000,170.400000,170.400000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,2015-04-24,-1.0,0
145,442,100001,2015-04-26,G,

In [ ]:
failure = pd.DataFrame(data.groupby(["TARGET_FAILURE"])["ID"].agg(["count"]));
failure["%"] = round(failure["count"]/failure["count"].sum()*100, 2);
failure

,count,%
TARGET_FAILURE,,
0,296011,96.19
1,11740,3.81


We have now improved our imbalance dataset a little bit by setting the target_failure within a "window" rather than a specific date. This is also useful for us because in reality, maintenance should be done ahead of time within a "window" rather than on the day of failure itself.

However, 3.8% of failure data is still relatively low for the whole dataset, so we can synthetically populate these failure data through statistical method called SMOTE, but we will perform it only on the training set. So we will have to perform splits on the dataset for training, validation and test sets. 

In [ ]:
#savepoint! 

data.to_pickle(path / "data_with_target_failure")

In [ ]:
# load picke
data = pd.read_pickle(path / "data_with_target_failure")

# Training, Cross Validation and Test Sets

Because the dataset came from the panel data (ie cross-sectional time series data, uniquely identified by machine `ID`), we need to avoid splitting the records from one single machine in all three sample datasets, in other words we should not randomize the sample of the records.

On the other hand, we randomly select IDs and place each of those machines in either training, validation or test sets.

In [ ]:
id_df = pd.DataFrame(data["ID"].unique(), columns = ["ID"])
id_df

,ID
0,100001
1,100002
2,100003
3,100006
4,100007
...,...
416,100612
417,100613
418,100614
419,100616


In [ ]:
# assign a random uniform probability to the machine ID's 
np.random.seed(42)
id_df["probability"] = (np.random.randint(0., 10000., id_df.shape[0]))/10000  # 10000 to make it floating point?

In [ ]:
id_df

,ID,probability,MODELING_SET
0,100001,0.7270,TEST
1,100002,0.0860,TRAINING
2,100003,0.5390,VALIDATION
3,100006,0.5191,VALIDATION
4,100007,0.5734,VALIDATION
...,...,...,...
416,100612,0.4070,VALIDATION
417,100613,0.9662,TEST
418,100614,0.0580,TRAINING
419,100616,0.7004,TEST


Let's give 35% of the probability to be the training set, 30% validation set and 35% test set.

In [ ]:
id_df["MODELING_SET"] = np.where( (id_df["probability"] <= 0.35), "TRAINING", np.where( (id_df["probability"] <= 0.65), "VALIDATION", "TEST" ) )

In [ ]:
id_df

,ID,probability,MODELING_SET
0,100001,0.7270,TEST
1,100002,0.0860,TRAINING
2,100003,0.5390,VALIDATION
3,100006,0.5191,VALIDATION
4,100007,0.5734,VALIDATION
...,...,...,...
416,100612,0.4070,VALIDATION
417,100613,0.9662,TEST
418,100614,0.0580,TRAINING
419,100616,0.7004,TEST


In [ ]:
id_sets = pd.DataFrame(id_df.groupby(["MODELING_SET"])["ID"].agg(["count"]));
id_sets["%"] = round(id_sets["count"]/id_sets["count"].sum()*100, 2);
id_sets

,count,%
MODELING_SET,,
TEST,149,35.39
TRAINING,146,34.68
VALIDATION,126,29.93


Let's merge `id_df` to our main `data`

In [ ]:
# sort the dataframes by "ID" first

id_df.sort_values(by=["ID"], ascending=True, inplace = True)
data.sort_values(by=["ID"], ascending=True, inplace= True)

In [ ]:
data = data.merge(id_df, how='left', on=['ID'])
data.tail(140)

,index,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT,flipper,START_DATE,TIME_SINCE_START,within_window,S15_mean,S17_mean,S13_mean,S5_mean,S16_mean,S19_mean,S18_mean,S8_mean,S15_median,S15_max,S15_min,S17_median,S17_max,S17_min,S13_median,S13_max,S13_min,S5_median,S5_max,S5_min,S16_median,S16_max,S16_min,S19_median,S19_max,S19_min,S18_median,S18_max,S18_min,S8_median,S8_max,S8_min,S15_change,S17_change,S13_change,S5_change,S16_change,S19_change,S18_change,S8_change,FAILURE_DATE,TIME_TO_FAILURE,TARGET_FAILURE,probability,MODELING_SET
307611,307611,100617,2015-09-23,C,P,S,7,5.6400,156.94,56.75,3802.0,9.6426,2.9,3.6,0,4.30,375,0,2014-12-02,295.0,0,5.6400,156.94,56.75,3802.0,9.6426,2.9,3.6,4.30,5.6400,5.6400,5.6400,156.94,156.94,156.94,56.75,56.75,56.75,3802.0,3802.0,3802.0,9.6426,9.6426,9.6426,2.9,2.9,2.9,3.6,3.6,3.6,4.30,4.30,4.30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016-09-21,364.0,0,0.813,TEST
307612,307612,100617,2015-09-02,C,P,S,7,6.1632,117.92,59.54,3989.0,8.3764,0.4,3.8,0,5.19,354,0,2014-12-02,274.0,0,6.1632,117.92,59.54,3989.0,8.3764,0.4,3.8,5.19,6.1632,6.1632,6.1632,117.92,117.92,117.92,59.54,59.54,59.54,3989.0,3989.0,3989.0,8.3764,8.3764,8.3764,0.4,0.4,0.4,3.8,3.8,3.8,5.19,5.19,5.19,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016-09-21,385.0,0,0.813,TEST
307613,307613,100617,2015-10-16,C,P,S,7,15.0336,70.30,61.73,4136.0,8.1648,2.9,12.1,0,14.14,398,0,2014-12-02,318.0,0,15.0336,70.30,61.73,4136.0,8.1648,2.9,12.1,14.14,15.0336,15.0336,15.0336,70.30,70.30,70.30,61.73,61.73,61.73,4136.0,4136.0,4136.0,8.1648,8.1648,8.1648,2.9,2.9,2.9,12.1,12.1,12.1,14.14,14.14,14.14,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016-09-21,341.0,0,0.813,TEST
307614,307614,100617,2015-10-18,C,P,S,7,7.0488,95.94,63.31,4242.0,8.4564,2.9,4.6,0,8.01,400,0,2014-12-02,320.0,0,7.0488,95.94,63.31,4242.0,8.4564,2.9,4.6,8.01,7.0488,7.0488,7.0488,95.94,95.94,95.94,63.31,63.31,63.31,4242.0,4242.0,4242.0,8.4564,8.4564,8.4564,2.9,2.9,2.9,4.6,4.6,4.6,8.01,8.01,8.01,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016-09-21,339.0,0,0.813,TEST
307615,307615,100617,2015-11-11,C,P,S,7,21.3600,0.00,45.88,3074.0,11.6880,2.9,10.6,0,8.74,424,0,2014-12-02,344.0,0,21.3600,0.00,45.88,3074.0,11.6880,2.9,10.6,8.74,21.3600,21.3600,21.3600,0.00,0.00,0.00,45.88,45.88,45.88,3074.0,3074.0,3074.0,11.6880,11.6880,11.6880,2.9,2.9,2.9,10.6,10.6,10.6,8.74,8.74,8.74,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2016-09-21,315.0,0,0.813,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307746,307746,100617,2015-08-04,C,P,S,7,4.7808,117.45,46.67,3127.0,9.5648,0.7,2.5,0,2.14,325,0,2014-12-02,245.0,0,4.7808,117.45,46.67,3127.0,9.5648,0.7,2.5,2.14,4.7808,4.7808,4.7808,117.45,117.45,117.45,46.67,46.67,46.67,3127.0,3127.0,3127.0,9.5648,9.5648,9.5648,0.7,0.7,0.7,2.5,2.5,2.5,2.14,2.14,2.14,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016-09-21,414.0,0,0.813,TEST
307747,307747,100617,2015-08-05,C,P,S,7,5.6448,110.70,47.88,3208.0,11.1264,0.7,2.4,0,2.63,326,0,2014-12-02,246.0,0,5.6448,110.70,47.88,3208.0,11.1264,0.7,2.4,2.63,5.6448,5.6448,5.6448,110.70,110.70,110.70,47.88,47.88,47.88,3208.0,3208.0,3208.0,11.1264,11.1264,11.1264,0.7,0.7,0.7,2.4,2.4,2.4,2.63,2.63,2.63,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016-09-21,413.0,0,0.813,TEST
307748,307748,100617,2015-08-06,C,P,S,7,4.4352,143.10,46.63,3124.0,8.4738,0.7,2.4,0,2.01,327,0,2014-12-02,247.0,0,4.4352,143.10,46.63,3124.0,8.4738,0.7,2.4,2.01,4.4352,4.4352,4.4352,143.10,143.10,143.10,46.63,46.63,46.63,3124.0,3124.0,3124.0,8.4738,8.4738,8.4738,0.7,0.7,0.7,2.4,2.4,2.4,2.01,2.01,2.01,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016-09-21,412.0,0,0.813,TEST
307749,307749,100617,2015-08-08,C,P,S,7,5.5872,167.50,46.45,3112.0,7.3200,0.4,2.2,0,2.09,329,0,2014-12-02,249.0,0,5.5872,167.50,46.45,3112.0,7.3200,0.4,2.2,2.09,5.5872,5.5872,5.5872,

Let's see how many records for each modeling sets:

In [ ]:
id_sets = pd.DataFrame(data.groupby(["MODELING_SET"])["ID"].agg(["count"]));
id_sets["%"] = round(id_sets["count"]/id_sets["count"].sum()*100, 2);
id_sets

,count,%
MODELING_SET,,
TEST,108919,35.39
TRAINING,106726,34.68
VALIDATION,92106,29.93


In [ ]:
data.groupby(["MODELING_SET"])["TARGET_FAILURE"].sum()

MODELING_SET
TEST          4151
TRAINING      4071
VALIDATION    3518
Name: TARGET_FAILURE, dtype: int64

Create separate dataframe for train, validation and test sets

In [ ]:
data.shape

(307751, 66)

In [ ]:
data.columns

Index(['index', 'ID', 'DATE', 'REGION_CLUSTER', 'MAINTENANCE_VENDOR',
       'MANUFACTURER', 'WELL_GROUP', 'S15', 'S17', 'S13', 'S5', 'S16', 'S19',
       'S18', 'EQUIPMENT_FAILURE', 'S8', 'AGE_OF_EQUIPMENT', 'flipper',
       'START_DATE', 'TIME_SINCE_START', 'within_window', 'S15_mean',
       'S17_mean', 'S13_mean', 'S5_mean', 'S16_mean', 'S19_mean', 'S18_mean',
       'S8_mean', 'S15_median', 'S15_max', 'S15_min', 'S17_median', 'S17_max',
       'S17_min', 'S13_median', 'S13_max', 'S13_min', 'S5_median', 'S5_max',
       'S5_min', 'S16_median', 'S16_max', 'S16_min', 'S19_median', 'S19_max',
       'S19_min', 'S18_median', 'S18_max', 'S18_min', 'S8_median', 'S8_max',
       'S8_min', 'S15_change', 'S17_change', 'S13_change', 'S5_change',
       'S16_change', 'S19_change', 'S18_change', 'S8_change', 'FAILURE_DATE',
       'TIME_TO_FAILURE', 'TARGET_FAILURE', 'probability', 'MODELING_SET'],
      dtype='object')

In [ ]:
# setting up training dataset
df_train = data[data["MODELING_SET"] == "TRAINING"]
df_train = df_train.drop(columns = ["MODELING_SET", "probability", "START_DATE", "flipper", "TIME_TO_FAILURE"])
df_train.shape

(106726, 61)

In [ ]:
# create separate dataframe for training and test datasets. We will use this to tweak our models
df_train_test = data[data["MODELING_SET"] != "VALIDATION"]
df_train_test= df_train_test.drop(columns = ["MODELING_SET", "probability", "START_DATE", "flipper", "TIME_TO_FAILURE"])
df_train_test.shape

(215645, 61)

In [ ]:
# create a separate dataframe for all the data. We will use this to validate the model and compare accuracy for all groups
df_total = data.drop(columns=["MODELING_SET", "probability", "START_DATE", "flipper", "TIME_TO_FAILURE"])
df_total.shape

(307751, 61)

# Synthetic Minority Oversampling Technique (SMOTE)